In [ ]:
import pandas as pd
import numpy as np
import time

import torch
from transformers import TapasTokenizer, TapasConfig, TapasForQuestionAnswering, AdamW


In [ ]:
def acc(num):
    from numpy.random import randint
    from random import sample
    
    encounter_list = [436, 236, 337, 791, 219, 574, 10, 555, 776, 262, 987, 352, 824, 602, 849, 198, 195, 
                      419, 286, 909, 107, 167, 169, 254, 650, 943, 838, 223, 685, 591, 277, 832, 283, 320, 
                      502, 905, 489, 874, 63, 425, 772, 991, 76, 878, 984, 990, 106, 451, 999, 748,627,196,197,
                      437, 212, 255, 666, 777, 888, 555, 323, 997, 199, 233]
    encounter_set = set(encounter_list)
    
    question_list = ["when was patient's first recorded time?","what was the last recorded time for patient",
                 "What was the patient's greatest heart rate?",
                'Among all the pulse record, what is the least pulse rate?','what is the greatest value for SpO2?', 
                 'what is the minimum value for SpO2?',
                "for this patient, what is the maximal respiration value?", "What's the minimum respiration rate?", 
                 "what was the most recent record for patient?", 
                 'what was the first time to measure the heart rate for the patient?',
                "For pulse rate, what is the greatest value?", "What is the least value of heart rate for the patient?", 
                 'what is the smallest MAP (mmHg) value?',
                'what is the largest MAP (mmHg) value?', 'what is the least temperature for this patient?', 
                 'what is the greatest temperature?',
                "At what time, the patient had the greatest heart rate?", 
                 'when was the least pulse rate?', 'at what time, this patient had the max SpO2?',
                 
                'when did the patient had his least SpO2?', "what time the patient, greatest respiration rate?", 
                 "what the least respiration rate time?",
                "What's the maximal pulse rate time?", "what was the least heart rate time?", 
                 'when was the minimum MAP (mmHg)?',
                'what was the biggest MAP (mmHg) time?', 'when was the minimum temperature for this patient?',
                 'greatest, what was the temperature time?']
    
    predicted_wrong_question = []
    answer_location = []
    correct_prediction = 0
    num_question = 0
    
    i = 0
    while i < num:
        csv_num = randint(1, 1001)
        if csv_num in encounter_list:
            continue
        else:
            
            question = sample(question_list,1)
            xx = pd.read_csv(f'dataset/{csv_num}.csv')
            inputs = tokenizer(table=xx.astype(str), queries=question, padding='max_length', return_tensors="pt")
            logits = compute_prediction_sequence(model, inputs, device)
            predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())
            predicted_answer_coordinates = predicted_answer_coordinates[0]

                
            if question == "when was patient's first recorded time?":
                q = [(0,0)]
                

            elif question == "what was the last recorded time for patient":
                locations = len(xx)-1
                q = [(locations,0)]
                print(xx)
               

            elif question == "What was the patient's greatest heart rate?":
                highest = max(xx['Heart Rate'])
                row_number = xx[xx['Heart Rate']==highest].index.values
                nums_rows = len(row_number)
                q = [(i,3) for i in row_number]
                print(xx)
              

            elif question == 'Among all the pulse record, what is the least pulse rate?':
                lowest = min(xx['Pulse'])
                row_number = xx[xx['Pulse']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,6) for i in row_number]
                print(xx)
                

            elif question == 'what is the greatest value for SpO2?':
                highest = max(xx['SpO2'])
                row_number = xx[xx['SpO2']==highest].index.values
                nums_rows = len(row_number)
                q = [(i,8) for i in row_number]
                print(xx)
        


            elif question == 'what is the minimum value for SpO2?':
                lowest = min(xx['SpO2'])
                row_number = xx[xx['SpO2']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,8) for i in row_number]
                print(xx)
     


            elif question == "for this patient, what is the maximal respiration value?":
                highest = max(xx['Resp'])
                row_number = xx[xx['Resp']==highest].index.values
                nums_rows = len(row_number)
                q = [(i,7) for i in row_number]
                print(xx)
         


            elif question == "What's the minimum respiration rate?":
                lowest = min(xx['Resp'])
                row_number = xx[xx['Resp']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,7) for i in row_number]
                print(xx)
       


            elif question == 'what was the first time to measure the heart rate for the patient?':
                q = [(0,0)]
          

            elif question == "what was the most recent record for patient?":
                q = [(len(xx)-1,0)]
                print(xx)



            elif question == "For pulse rate, what is the greatest value?":
                highest = max(xx['Pulse'])
                row_number = xx[xx['Pulse']==highest].index.values
                nums_rows = len(row_number)
                q = [(i,6) for i in row_number]
                print(xx)
           


            elif question == "What is the least value of heart rate for the patient?":
                lowest = min(xx['Heart Rate'])
                row_number = xx[xx['Heart Rate']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,3) for i in row_number]
                print(xx)
    


            elif question == 'what is the smallest MAP (mmHg) value?':
                lowest = min(xx['MAP (mmHg)'])
                row_number = xx[xx['MAP (mmHg)']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,5) for i in row_number]
                print(xx)
             


            elif question == 'what is the largest MAP (mmHg) value?':
                highest = max(xx['MAP (mmHg)'])
                row_number = xx[xx['MAP (mmHg)']==highest].index.values
                nums_rows = len(row_number)
                q = [(i,5) for i in row_number]
                print(xx)
          


            elif question == 'what is the least temperature for this patient?':
                lowest = min(xx['Temp'])
                row_number = xx[xx['Temp']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,9) for i in row_number]
                print(xx)
           


            elif question == 'what is the greatest temperature?':
                highest = max(xx['Temp'])
                row_number = xx[xx['Temp']==highest].index.values
                nums_rows = len(row_number)
                q = [(i,9) for i in row_number]
                print(xx)
           



            elif question == "At what time, the patient had the greatest heart rate?":
                highest = max(xx['Heart Rate'])
                row_number = xx[xx['Heart Rate']==highest].index.values
                nums_rows = len(row_number)
                q =[(i,0) for i in row_number]
                print(xx)
              


            elif question == 'when was the least pulse rate?':
                lowest = min(xx['Pulse'])
                row_number = xx[xx['Pulse']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,0) for i in row_number]
                print(xx)
     


            elif question == 'at what time, this patient had the max SpO2?':
                highest = max(xx['SpO2'])
                row_number = xx[xx['SpO2']==highest].index.values
                nums_rows = len(row_number)
                q = [(i,0) for i in row_number]
                print(xx)
           


            elif question == 'when did the patient had his least SpO2?':
                lowest = min(xx['SpO2'])
                row_number = xx[xx['SpO2']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,0) for i in row_number]
                print(xx)
            


            elif question == "what time the patient, greatest respiration rate?":
                highest = max(xx['Resp'])
                row_number = xx[xx['Resp']==highest].index.values
                nums_rows = len(row_number)
                q = [(i,0) for i in row_number]
                print(xx)
                


            elif question == "what the least respiration rate time?":
                lowest = min(xx['Resp'])
                row_number = xx[xx['Resp']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,0) for i in row_number]
                print(xx)
          


            elif question == "What's the maximal pulse rate time?":
                highest = max(xx['Pulse'])
                row_number = xx[xx['Pulse']==highest].index.values
                nums_rows = len(row_number)
                q = [(i,0) for i in row_number]
                print(xx)
             


            elif question == "what was the least heart rate time?":
                lowest = min(xx['Heart Rate'])
                row_number = xx[xx['Heart Rate']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,0) for i in row_number]
                print(xx)
                


            elif question == 'when was the minimum MAP (mmHg)?':
                lowest = min(xx['MAP (mmHg)'])
                row_number = xx[xx['MAP (mmHg)']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,0) for i in row_number]
                print(xx)
              


            elif question == 'what was the biggest MAP (mmHg) time?':
                highest = max(xx['MAP (mmHg)'])
                row_number = xx[xx['MAP (mmHg)']==highest].index.values
                nums_rows = len(row_number)
                q = [(i,0) for i in row_number]
                print(xx)
               


            elif question == 'when was the minimum temperature for this patient?':
                lowest = min(xx['Temp'])
                row_number = xx[xx['Temp']==lowest].index.values
                nums_rows = len(row_number)
                q = [(i,0) for i in row_number]
               


            else: 
                highest = max(xx['Temp'])
                row_number = xx[xx['Temp']==highest].index.values
                nums_rows = len(row_number)
                q = [(i,0) for i in row_number]
                print(xx)
              

            i += 1
            num_question +=1

            if q == predicted_answer_coordinates:
                    correct_prediction += 1
            else:
                predicted_wrong_question.append(question)